In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import faiss 
import os
import numpy as np
from accelerate import init_empty_weights

In [6]:
#bits and bytes not optimized on windows
#import bitsandbytes as bnb
# print("Bitsandbytes version:", bnb.__version__)
# print("CUDA Available:", bnb.is_available())
# from bitsandbytes.cuda_setup.main import get_compute_capabilities, get_cuda_version

In [ ]:
# print("Cache directory:", os.getenv("TRANSFORMERS_CACHE"))
# print("Cache directory:", os.getenv("HF_HOME"))

In [7]:
# print(bnb.__version__)
# print(torch.__version__)
# print(torch.version.cuda)
# print("Detected CUDA Version (bitsandbytes):", get_cuda_version())
# print("Compute Capabilities:", get_compute_capabilities())
#print("Bitsandbytes CUDA Version:", bnb.nvidia_driver_version())

In [8]:
#4-bit quantization 
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

In [14]:
pre_quantized_model="TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"

In [18]:
#check if model supports quantization
model_name = "mistralai/Mistral-7B-v0.1"  # Change this to your model
model = AutoModelForCausalLM.from_pretrained(model_name)
print("Model supports quantization:", hasattr(model, "quantization_config"))

d:\anaconda3\envs\llm_rag_env\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\achen\.cache\huggingface\hub\models--mistralai--Mistral-7B-v0.1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.11s/it]


Model supports quantization: False


In [21]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Check if remote code is required
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
print("Model Loaded Successfully!")


Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.65s/it]


Model Loaded Successfully!


In [12]:
model_name="mistralai/Mistral-7B-Instruct-v0.1"
device="cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
tokenizer=AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model=AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.float16, 
    device_map="auto",
    offload_folder="D:/LLM_Projects/offload",
    #quantization_config=bnb_config,
    #trust_remote_code=True
    
)

prompt="Explain what a model is"

inputs=tokenizer(prompt, return_tensors="pt")
inputs={key:value.to(device) for key, value in inputs.items()}
with torch.no_grad():
    output=model.generate(**inputs, max_length=100)


Using device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [13]:
print(tokenizer.decode(output[0]))

<s> Explain what a model is in the context of machine learning.
A model in the context of machine learning is a mathematical or algorithmic representation of a system, process or relationship between variables that can be used to make predictions or decisions. It is created by training a machine learning algorithm on a dataset, which allows the algorithm to learn patterns and make inferences about new data. The model can then be used to make predictions or decisions based on new input data. The accuracy and effectiveness of a
